In [12]:
import pandas as pd
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
from tqdm import tqdm 
from urllib.parse import urljoin
pd.set_option('display.max_rows', 500)


In [3]:
def data_prep(subset:bool = False) -> pd.DataFrame:
    db_pomoci = pd.read_csv("../db_pomoci.csv")
    df = db_pomoci.rename(columns={"Webová stránka" : "web", "Název":"nazev","E-mail":"email"})
    black_list = ['http://www.dc-brno.cz','www.freeklub.cz', 'http://www.cszs.cz/','http://www.fokustabor.cz/centrum-dusevniho-zdravi-_-komunitni-tym-tabor']
    df = df[(~df["web"].isna()) & (~df['web'].isin(black_list))]
    df['web'] = df['web'].str.replace(r'\s+', '', regex=True)
    df.loc[df['web'].str.contains(r'\.cz/.+'), 'web'] = df['web'].str.replace(r'\.cz/.+', '.cz', regex=True)
    df.loc[df['web'].str.startswith('www'), 'web'] = df['web'].str.replace('^www', 'https://www', regex=True)
    df_agg_tel = df.groupby("web")["Telefon"].agg(list).reset_index()
    df_agg_email = df.groupby("web")["email"].agg(list).reset_index()
    df_agg = pd.merge(df_agg_tel, df_agg_email, on="web")#.drop("index", axis= 1)
    df_agg = df_agg.rename(columns={'web': 'web', 'Telefon': 'Telefon'})
    df_agg = pd.DataFrame(df_agg)
    if subset:
        df_agg = df_agg.head(100) # ubset first 300 rows
    return df_agg

In [17]:
# db_pomoci = pd.read_csv("db_pomoci.csv")

# black_list = ['http://www.dc-brno.cz','www.freeklub.cz, www.cszs.cz','https://www.freeklub.cz']

# df = db_pomoci.rename(columns={"Webová stránka" : "web", "Název":"nazev","E-mail":"email"})

# df = df[(~df["web"].isna()) & (~df['web'].isin(black_list))]

# df.loc[df['web'].str.startswith('www'), 'web'] = df['web'].str.replace('^www', 'https://www', regex=True)

# df_agg_tel = df.groupby("web")["Telefon"].agg(list).reset_index()
# df_agg_email = df.groupby("web")["email"].agg(list).reset_index()
# df_agg = pd.merge(df_agg_tel, df_agg_email, on="web")#.drop("index", axis= 1)
# df_agg = df_agg.rename(columns={'web': 'web', 'Telefon': 'Telefon'})
# df_agg = pd.DataFrame(df_agg)
# df_agg = df_agg[~df_agg['web'].isin(black_list)]
# df_agg.head()
# #df.sort_values("web")


In [4]:
def scrape_urls(df):    
    email_regex = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}")
    #Phone regex V2
    phone_regex = re.compile(r"""
        (?:
            \+420[-\s]?        # Optional country code +420 followed by an optional space or dash
        )?
        (?:
            \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
            \d{3}[-\s]?        # Second part of the phone number (3 digits) followed by an optional space or dash
            \d{3}              # Third part of the phone number (3 digits)
            |                  # OR
            \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
            \d{2}[-\s]?        # Second part of the phone number (2 digits) followed by an optional space or dash
            \d{2}[-\s]?        # Third part of the phone number (2 digits) followed by an optional space or dash
            \d{2}              # Fourth part of the phone number (2 digits)
        )
    """, re.VERBOSE)
    #Phone regex V1
    # phone_regex = re.compile(r"""
    #     (?:
    #         \+420[-\s]?       # Optional country code +420 followed by an optional space or dash
    #     )?
    #     (?:                  # Non-capturing group for the phone number
    #         \d{3}[-\s]?      # First part of the phone number (3 digits)
    #         \d{3}[-\s]?      # Second part of the phone number (3 digits)
    #         \d{3}            # Third part of the phone number (3 digits)
    #         (?:\d{3})?       # Optional fourth part of the phone number (3 digits)
    #     )
    # """, re.VERBOSE)

    # List to collect data
    data = []

    # Function to scrape a single URL
    def scrape_website_main_page(url):
        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            emails = set(email_regex.findall(soup.text))
            phones = set(phone_regex.findall(soup.text))
            return emails, phones, url
        except requests.RequestException as e:
            return set(), set(), url

    # Function to scrape potential contact pages
    def scrape_website_contacts(base_url, path='', visited_urls=set()):
        full_url = urljoin(base_url, path)
        if full_url in visited_urls:
            return set(), set(), full_url  # Return empty sets if URL has already been visited
        visited_urls.add(full_url)

        try:
            response = requests.get(full_url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            emails = set(email_regex.findall(soup.text))
            phones = set(phone_regex.findall(soup.text))

            # Checking for links that may contain contact info
            contact_links = []
            for link in soup.find_all('a', href=True):
                if 'kontakt' in link.text.lower() \
                    or 'contact' in link.text.lower() \
                    or 'kontakty' in link.text.lower() \
                    or 'kdo-jsem' in link.text.lower() \
                    or 'kontakt ' in link.text.lower() \
                    or 'o-nas' in link.text.lower():
                    contact_href = urljoin(full_url, link['href'])
                    if contact_href not in visited_urls:
                        contact_links.append(contact_href)

            # Scrape each potential contact page
            for link in contact_links:
                sub_emails, sub_phones, sub_url = scrape_website_main_page(link)
                emails.update(sub_emails)
                phones.update(sub_phones)

            return emails, phones, full_url
        except requests.RequestException as e:
            return set(), set(), full_url

    # Set of URLs to scrape
    urls = set(df["web"])

    # Loop through the URLs and scrape each one
    visited_urls = set()
    for url in tqdm(urls):
        emails, phones, page_url = scrape_website_contacts(url, visited_urls=visited_urls)
        emails_str = ', '.join(emails)
        phones_str = ', '.join(phones)
        
        # Collect data
        data.append({'Base Website': url, 'Scraped Page': page_url, 'Emails': emails_str, 'Phone Numbers': phones_str})

    return data
    # def scrape_website_contacts(base_url, path='', visited_urls=set()):
    #     full_url = urljoin(base_url, path)
    #     if full_url in visited_urls:
    #         return set()  # Return empty set if URL has already been visited
    #     visited_urls.add(full_url)

    #     try:
    #         response = requests.get(full_url)
    #         response.raise_for_status()
    #         soup = BeautifulSoup(response.text, 'html.parser')

    #         emails = set(email_regex.findall(soup.text))
    #         phones = set(phone_regex.findall(soup.text))

    #         # Checking for links that may contain contact info
    #         contact_links = []
    #         for link in soup.find_all('a', href=True):
    #             if 'kontakt' in link.text.lower() \
    #                 or 'contact' in link.text.lower() \
    #                 or 'kontakty' in link.text.lower() \
    #                 or 'kdo-jsem' in link.text.lower() \
    #                 or 'o-nas' in link.text.lower():
    #                 contact_href = urljoin(full_url, link['href'])
    #                 if contact_href not in visited_urls:
    #                     contact_links.append(contact_href)

    #         # Scrape each potential contact page
    #         for link in contact_links:
    #             sub_emails, sub_phones = scrape_website_main_page(link)
    #             emails.update(sub_emails)
    #             phones.update(sub_phones)

    #         return emails, phones
    #     except requests.RequestException as e:
    #         #print(f"Error fetching {full_url}: {e}")
    #         return set(),set()

    

    # Loop through the URLs and scrape each one
    # visited_urls = set()
    # for url in tqdm(urls):
    #     #print(f"Scraping {url}")
        
    #     emails, phones = scrape_website_contacts(url, visited_urls=visited_urls)
    #     ###
    #     # Condition if main page does not contain contact infromation
    #     ###    
    #     # if  ('' in emails) or  ('' in phones):        
    #     #     emails_2, phones_2 = scrape_website_contacts(url, visited_urls=visited_urls)
    #     #     emails.add(emails_2)
    #     #     phones.add(phones_2)
    #     emails_str = ', '.join(emails)
    #     phones_str = ', '.join(phones)
        
    #     # Collect data
    #     data.append({'Website': url, 'Emails': emails_str, 'Phone Numbers': phones_str})
    #     #print("Done")
    #     #print("")
    # return data


In [15]:
df = data_prep(subset=False)
#total_rows = df.shape[0]
df = df.loc[300:600]
df.shape

(301, 3)

In [16]:
df_res = pd.DataFrame(scrape_urls(df))


 92%|█████████▏| 277/301 [11:21<00:59,  2.46s/it]  


KeyboardInterrupt: 

In [7]:
df_res.to_csv('../data/29_05/df_scraped_v6_100.csv', index=False)

In [13]:
df_scraped_v2_300_600 = pd.read_csv('../data/29_05/df_scraped_v6_100.csv')
df_scraped_v2_300_600
# df_scraped_v2_300_600[(df_scraped_v2_300_600['Emails'].isna()) | (df_scraped_v2_300_600['Phone Numbers'].isna())]

,Base Website,Scraped Page,Emails,Phone Numbers
0,http://www.aura-psycholog.cz/,http://www.aura-psycholog.cz/,e.sonkova@gmail.com,724 135 213
1,http://psychologie-biofeedback.cz/,http://psychologie-biofeedback.cz/,"luciesmahelova@seznam.czskype, luciesmahelova@...","+420 607 038 554, 607 038 554, +420 608 863 008"
2,http://www.at-ambulance.cz/,http://www.at-ambulance.cz/,NaN,"+420 567 574 555, 977\n1996-20"
3,http://poradna.krumlov.cz,http://poradna.krumlov.cz,NaN,NaN
4,http://www.centrum-poradenstvi.cz,http://www.centrum-poradenstvi.cz,"poradna.uh@centrum-poradenstvi.cz, poradna.zli...","775 760 719, 777 219 150, 577 210 809, 777 219..."
5,http://www.bunkr.cz/,http://www.bunkr.cz/,NaN,"200014310, +420 734 178 563, +420 734 321 931,..."
6,http://pivecka-ops.cz,http://pivecka-ops.cz,"315njp@pivecka.cz, 408poradnazebra@gmail.com, ...","+420 739 095 315, +420 603 271 904, 185834029,..."
7,http://sela.webnode.cz/,http://sela.webnode.cz/,sela@o2active.cz,"723858018, 197183989, 723 858 018, 321722180, ..."
8,http://ppp-olomouc.cz/,http://ppp-olomouc.cz/,NaN,"585 224 573, 778 442 448, 585 221 045"
9,http://psychologslany.cz,http://psychologslany.cz,NaN,NaN


In [113]:
df = data_prep(subset=False)
(df.shape[0]-(129 + 121 + 138))/df.shape[0]


0.5519630484988453

In [115]:
df1 = pd.read_csv('data/29_05/df_scraped_v2_300.csv')
df2 = pd.read_csv('data/29_05/df_scraped_v2_300_600.csv')
df3 = pd.read_csv('data/29_05/df_scraped_v2_601_plus.csv')

# Concatenate the DataFrames
concatenated_df = pd.concat([df1, df2, df3], ignore_index=True)

# Save the concatenated DataFrame to a new CSV file
concatenated_df.to_csv('df_scraped_v2.csv', index=False)

(866, 3)

In [94]:
test_emails = ['cheiront@cheiront.cz']
email_regex = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}")

for email in test_emails:
    match = email_regex.search(email)
    if match:
        print(f"Matched: {match.group()}")
    else:
        print(f"No match: {email}")

Matched: cheiront@cheiront.cz


# Specific web check

In [ ]:
email_regex = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}")
#Phone regex V2
phone_regex = re.compile(r"""
    (?:
        \+420[-\s]?        # Optional country code +420 followed by an optional space or dash
    )?
    (?:
        \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
        \d{3}[-\s]?        # Second part of the phone number (3 digits) followed by an optional space or dash
        \d{3}              # Third part of the phone number (3 digits)
        |                  # OR
        \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
        \d{2}[-\s]?        # Second part of the phone number (2 digits) followed by an optional space or dash
        \d{2}[-\s]?        # Third part of the phone number (2 digits) followed by an optional space or dash
        \d{2}              # Fourth part of the phone number (2 digits)
    )
""", re.VERBOSE)
#Phone regex V1
# phone_regex = re.compile(r"""
#     (?:
#         \+420[-\s]?       # Optional country code +420 followed by an optional space or dash
#     )?
#     (?:                  # Non-capturing group for the phone number
#         \d{3}[-\s]?      # First part of the phone number (3 digits)
#         \d{3}[-\s]?      # Second part of the phone number (3 digits)
#         \d{3}            # Third part of the phone number (3 digits)
#         (?:\d{3})?       # Optional fourth part of the phone number (3 digits)
#     )
# """, re.VERBOSE)

# List to collect data
data = []

# Function to scrape a single URL
def scrape_website_main_page(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raises an HTTPError if the response status code is 4XX or 5XX
        soup = BeautifulSoup(response.text, 'html.parser')
        emails = set(email_regex.findall(soup.text))
        phones = set(phone_regex.findall(soup.text))
        return emails, phones
    except requests.RequestException as e:
        #print(f"Error fetching {url}: {e}")
        return set(),set()

# Function to scrape potential contact pages
def scrape_website_contacts(base_url, path='', visited_urls=set()):
    full_url = urljoin(base_url, path)
    if full_url in visited_urls:
        return set()  # Return empty set if URL has already been visited
    visited_urls.add(full_url)

    try:
        response = requests.get(full_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        emails = set(email_regex.findall(soup.text))
        phones = set(phone_regex.findall(soup.text))

        # Checking for links that may contain contact info
        contact_links = []
        for link in soup.find_all('a', href=True):
            if 'kontakt' in link.text.lower() \
                or 'contact' in link.text.lower() \
                or 'kontakty' in link.text.lower() \
                or 'kdo-jsem' in link.text.lower() \
                or 'o-nas' in link.text.lower():
                contact_href = urljoin(full_url, link['href'])
                if contact_href not in visited_urls:
                    contact_links.append(contact_href)

        # Scrape each potential contact page
        for link in contact_links:
            sub_emails, sub_phones = scrape_website_main_page(link)
            emails.update(sub_emails)
            phones.update(sub_phones)

        return emails, phones
    except requests.RequestException as e:
        #print(f"Error fetching {full_url}: {e}")
        return set(),set()
web = 'http://www.psychoterapie-iz.cz/index.php/kontakt'
print(scrape_website_main_page(web))

In [ ]:
dfr = pd.read_csv('../data/29_05/df_scraped_v3_300.csv')

dfr[dfr['Phone Numbers'].isna()].shape
# dfr[(dfr['Emails'].isna()) | (dfr['Phone Numbers'].isna())]


In [ ]:
dfr = pd.read_csv('../data/29_05/df_scraped_v5_300.csv')

dfr[dfr['Phone Numbers'].isna()].shape

In [76]:
def sublinks(base_url, path='', visited_urls=set()):
    full_url = urljoin(base_url, path)
    if full_url in visited_urls:
        return set()  # Return empty set if URL has already been visited
    visited_urls.add(full_url)
    
    print(f"Visiting: {full_url}")  # Debug statement

    try:
        response = requests.get(full_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')        

        # Checking for links that may contain contact info
        contact_links = []
        for link in soup.find_all('a', href=True):            
            contact_href = urljoin(full_url, link['href'])
            if contact_href not in visited_urls:
                contact_links.append(contact_href)
        
        print(f"Web {base_url} has {len(contact_links)} subpages")  # Debug statement

        sp = len(contact_links)
        return sp
    except requests.RequestException as e:
        print(f"Error accessing {full_url}: {e}")  # Debug statement
        return 0

In [ ]:
sp = sublinks(url, visited_urls=visited_urls)
sp

In [ ]:
urls = set(df["web"])

# Loop through the URLs and scrape each one
visited_urls = set()
total_pages = 0
total_subpages = 0
for url in urls:
    total_pages +=1
    print(f"Total pages count: {total_pages}")
    
    sp = sublinks(url, visited_urls=visited_urls)
    total_subpages = total_subpages + sp
    print(f"Total subpages count: {total_subpages}")
    print("#####################")

# Parallel Scraping 

In [1]:
import concurrent.futures

In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed
def scrape_urls(df):    
    email_regex = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}")
    phone_regex = re.compile(r"""
        (?:
            \+420[-\s]?        # Optional country code +420 followed by an optional space or dash
        )?
        (?:
            \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
            \d{3}[-\s]?        # Second part of the phone number (3 digits) followed by an optional space or dash
            \d{3}              # Third part of the phone number (3 digits)
            |                  # OR
            \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
            \d{2}[-\s]?        # Second part of the phone number (2 digits) followed by an optional space or dash
            \d{2}[-\s]?        # Third part of the phone number (2 digits) followed by an optional space or dash
            \d{2}              # Fourth part of the phone number (2 digits)
        )
    """, re.VERBOSE)

    def scrape_website_main_page(url):
        try:
            response = requests.get(url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            emails = set(email_regex.findall(soup.text))
            phones = set(phone_regex.findall(soup.text))
            return emails, phones
        except requests.RequestException as e:
            return set(), set()

    def scrape_website_contacts(base_url, path='', visited_urls=set()):
        full_url = urljoin(base_url, path)
        if full_url in visited_urls:
            return set(), set()
        visited_urls.add(full_url)

        try:
            response = requests.get(full_url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            emails = set(email_regex.findall(soup.text))
            phones = set(phone_regex.findall(soup.text))

            contact_links = []
            for link in soup.find_all('a', href=True):
                if any(keyword in link.text.lower() for keyword in ['kontakt', 'contact', 'kontakty', 'kdo-jsem', 'o-nas']):
                    contact_href = urljoin(full_url, link['href'])
                    if contact_href not in visited_urls:
                        contact_links.append(contact_href)

            for link in contact_links:
                sub_emails, sub_phones = scrape_website_main_page(link)
                emails.update(sub_emails)
                phones.update(sub_phones)

            return emails, phones
        except requests.RequestException as e:
            return set(), set()

    def scrape_website(url):
        visited_urls = set()
        emails, phones = scrape_website_contacts(url, visited_urls=visited_urls)
        emails_str = ', '.join(emails)
        phones_str = ', '.join(phones)
        return {'Website': url, 'Emails': emails_str, 'Phone Numbers': phones_str}

    urls = set(df["web"])

    data = []
    with ThreadPoolExecutor(max_workers=6) as executor:
        future_to_url = {executor.submit(scrape_website, url): url for url in urls}
        for future in tqdm(as_completed(future_to_url), total=len(urls)):
            url = future_to_url[future]
            try:
                result = future.result()
                data.append(result)
            except Exception as exc:
                print(f"{url} generated an exception: {exc}")

    return data

In [1]:
def scrape_urls(df):    
    email_regex = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}")
    #Phone regex V2
    phone_regex = re.compile(r"""
        (?:
            \+420[-\s]?        # Optional country code +420 followed by an optional space or dash
        )?
        (?:
            \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
            \d{3}[-\s]?        # Second part of the phone number (3 digits) followed by an optional space or dash
            \d{3}              # Third part of the phone number (3 digits)
            |                  # OR
            \d{3}[-\s]?        # First part of the phone number (3 digits) followed by an optional space or dash
            \d{2}[-\s]?        # Second part of the phone number (2 digits) followed by an optional space or dash
            \d{2}[-\s]?        # Third part of the phone number (2 digits) followed by an optional space or dash
            \d{2}              # Fourth part of the phone number (2 digits)
        )
    """, re.VERBOSE)
    #Phone regex V1
    # phone_regex = re.compile(r"""
    #     (?:
    #         \+420[-\s]?       # Optional country code +420 followed by an optional space or dash
    #     )?
    #     (?:                  # Non-capturing group for the phone number
    #         \d{3}[-\s]?      # First part of the phone number (3 digits)
    #         \d{3}[-\s]?      # Second part of the phone number (3 digits)
    #         \d{3}            # Third part of the phone number (3 digits)
    #         (?:\d{3})?       # Optional fourth part of the phone number (3 digits)
    #     )
    # """, re.VERBOSE)

    # List to collect data
    data = []

    # Function to scrape a single URL
    def scrape_website_main_page(url):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raises an HTTPError if the response status code is 4XX or 5XX
            soup = BeautifulSoup(response.text, 'html.parser')
            emails = set(email_regex.findall(soup.text))
            phones = set(phone_regex.findall(soup.text))
            return emails, phones
        except requests.RequestException as e:
            #print(f"Error fetching {url}: {e}")
            return set(),set()

    # Function to scrape potential contact pages
    def scrape_website_contacts(base_url, path='', visited_urls=set()):
        full_url = urljoin(base_url, path)
        if full_url in visited_urls:
            return set()  # Return empty set if URL has already been visited
        visited_urls.add(full_url)

        try:
            response = requests.get(full_url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')

            emails = set(email_regex.findall(soup.text))
            phones = set(phone_regex.findall(soup.text))

            # Checking for links that may contain contact info
            contact_links = []
            for link in soup.find_all('a', href=True):
                if 'kontakt' in link.text.lower() \
                    or 'contact' in link.text.lower() \
                    or 'kontakty' in link.text.lower() \
                    or 'kdo-jsem' in link.text.lower() \
                    or 'o-nas' in link.text.lower():
                    contact_href = urljoin(full_url, link['href'])
                    if contact_href not in visited_urls:
                        contact_links.append(contact_href)

            # Scrape each potential contact page
            for link in contact_links:
                sub_emails, sub_phones = scrape_website_main_page(link)
                emails.update(sub_emails)
                phones.update(sub_phones)

            return emails, phones
        except requests.RequestException as e:
            #print(f"Error fetching {full_url}: {e}")
            return set(),set()

    # Set of URLs to scrape
    urls = set(df["web"])

    # Loop through the URLs and scrape each one
    visited_urls = set()
    for url in tqdm(urls):
        #print(f"Scraping {url}")
        
        emails, phones = scrape_website_contacts(url, visited_urls=visited_urls)
        ###
        # Condition if main page does not contain contact infromation
        ###    
        # if  ('' in emails) or  ('' in phones):        
        #     emails_2, phones_2 = scrape_website_contacts(url, visited_urls=visited_urls)
        #     emails.add(emails_2)
        #     phones.add(phones_2)
        emails_str = ', '.join(emails)
        phones_str = ', '.join(phones)
        
        # Collect data
        data.append({'Website': url, 'Emails': emails_str, 'Phone Numbers': phones_str})
        #print("Done")
        #print("")
    return data


In [2]:
df_res = pd.DataFrame(scrape_urls(df))


NameError: name 'pd' is not defined

In [ ]:
df_res.to_csv('../data/29_05/df_scraped_paral.csv', index=False)
df_scraped_v2_300_600 = pd.read_csv('data/29_05/df_scraped_paral.csv')
df_scraped_v2_300_600[(df_scraped_v2_300_600['Emails'].isna()) | (df_scraped_v2_300_600['Phone Numbers'].isna())]

In [37]:
df1 = pd.read_csv('../data/1_06/df_scraped_1.csv')
df2 = pd.read_csv('../data/1_06/df_scraped_2.csv')
df3 = pd.read_csv('../data/1_06/df_scraped_3.csv')
df4 = pd.read_csv('../data/1_06/df_scraped_4.csv')


# Concatenate the DataFrames
concatenated_df = pd.concat([df1, df2, df3,df4], ignore_index=True)

# Save the concatenated DataFrame to a new CSV file
concatenated_df.to_csv('../data/1_06/df_scraped_full.csv', index=False)

In [39]:
df_scraped_full = pd.read_csv('../data/1_06/df_scraped_full.csv')
df_scraped_full.shape

(1882, 6)

In [21]:
#df_scraped_full.loc[df_scraped_full['Scraped Page'].str.endswith('kontakty')]
df_scraped_full.loc[df_scraped_full['Scraped Page'] != df_scraped_full['Scraped Page']]

,Unnamed: 0,Base Website,Scraped Page,Emails,Phone Numbers


In [36]:
response = requests.get("http://www.os-semiramis.cz")
response.raise_for_status()
soup = BeautifulSoup(response.text, 'html.parser')


# Checking for links that may contain contact info
contact_links = []
for link in soup.find_all('a', href=True):
    contact_links.append(link.text.lower())
    #print(link.text.lower())

contact_links

['',
 '',
 'o nás',
 'základní informace',
 'kde působíme',
 'podporují nás',
 'výroční zprávy',
 'podání stížnosti',
 'ochrana osobních údajů',
 'sociálně-právní ochrana',
 'ochrana práv klientů',
 'zaměstnání a praxe',
 'centra',
 'nízkoprahové zařízení pro děti a mládež n klub',
 'centrum primární prevence středočeského kraje',
 'centrum primární prevence královéhradeckého kraje',
 'centrum primární prevence pardubického kraje',
 'centrum rodinného poradenství a terapie',
 'terénní program semiramis',
 'kontaktní a poradenské centrum mladá boleslav',
 'centrum vzdělávacích aktivit',
 'vzdělávání',
 'adaptační kurzy',
 'intervenční programy',
 'kontakty',
 '',
 '\n\n\n\n\n\n\n\n',
 '\n\t\t\t\t\t\t   \n\t\t\t\t\t\tslužby pro děti a dospívající\n\t\t\t\t\t\t\t\t\t\t\t\t',
 '\n\n\n\n\n\n\n\n',
 '\n\t\t\t\t\t\t   \n\t\t\t\t\t\tslužby pro školy a pedagogy\n\t\t\t\t\t\t\t\t\t\t\t\t',
 '\n\n\n\n\n\n\n\n',
 '\n\t\t\t\t\t\t   \n\t\t\t\t\t\tslužby pro rodiny\n\t\t\t\t\t\t\t\t\t\t\t\t',
 '\n\n\

In [ ]:
'kontakt' in link.text.lower() \
                or 'contact' in link.text.lower() \
                or 'kontakty' in link.text.lower() \
                or 'kdo-jsem' in link.text.lower() \
                or 'kontakt a objednání' in link.text.lower() \
                or 'o-nas' in link.text.lower():